# Spotify Playlist Data Set


In [ ]:
import spotipy
import spotipy.util as util
#from config import CLIENT_ID, CLIENT_SECRET, PLAY_LIST, USER
import random
import requests
import time
import sys 
import os 
import json
import pandas as pd 

## Launch Spotipy API
- Get playlists made by Spotify  

In [ ]:
token = util.oauth2.SpotifyClientCredentials(client_id='d29d4c8d6bca47c4a890e95a276e37ac', client_secret='a5a967a13a2c4ffdb19d1bb8f8fe2622')
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)
user = 'spotify'
playlists = sp.user_playlists(user)

# get url of each playlist
request_urls = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        request_urls.append(playlist.get('href'))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


## Scrape data from each playlist

In [ ]:
def get_playlist_info(playlist,user = 'spotify'):
    """
    Get all available data from each playlist 
    """
    
    if 'https' in playlist:
        playlist = os.path.basename(playlist)
    
    # Query playlist 
    results = sp.user_playlist(user,playlist)
    
    # get meta-data
    play_keys = ['description', 'collaborative', 'public', 'snapshot_id']
    playlist_data = {key: value for key, value in results.items() if key in play_keys}
    playlist_data['followers'] = results['followers']['total']
    playlist_data['playlist_uri'] = results['uri']
    playlist_data['playlist_name'] = results['name']
    playlist_data['playlist_href'] = results['href']
    playlist_data['playlist_id'] = results['id']
    
    # get song data 
    tracks = []
    track_keys = ['explicit','duration_ms','track_number','popularity']
    feat_keys = ['liveness','acousticness','instrumentalness', \
                 'key','energy','loudness','time_signature','tempo', \
                 'speechiness','mode','valence','danceability']
    track_ids = []
    for track in results['tracks']['items']:
        track_id = track['track']['id']
        track_ids.append(track_id)
        track_dict = {key: value for key, value in track['track'].items() if key in track_keys}
        track_dict['track_href'] = track['track']['href']
        track_dict['track_uri'] = track['track']['uri']
        track_dict['track_name'] = track['track']['name']
        track_dict['track_id'] = track_id

        # include featured artists at a later date
        track_dict['artist'] = track['track']['artists'][0]['name']
        track_dict['artist_id'] = track['track']['artists'][0]['id']
        track_dict['artist_href'] = track['track']['artists'][0]['href']
        track_dict['artist_uri'] = track['track']['artists'][0]['uri']
        track_dict['album'] = track['track']['album']['name']
        track_dict['album_id'] = track['track']['album']['id']
        track_dict['album_href'] = track['track']['album']['href']
        track_dict['album_uri'] = track['track']['album']['uri']
        
        # append to dict
        tracks.append(track_dict)
        del track_dict

    # get audio features for each track 
    if len(track_ids) <= 50: # maximum number of songs per request 
        features = sp.audio_features(track_ids)
    else:
        features = []
        subList = [track_ids[n:n+50] for n in range(0, len(track_ids), 50)]
        for subset in subList:
            feat = sp.audio_features(subset)
            features.extend(feat)
            time.sleep(1)
        
    for ii,feat in enumerate(features):
        if feat is None:
            feat = {'acousticness': None,'danceability': None,'duration_ms': None,'energy': None, \
             'instrumentalness': None,'key': None,'liveness': None,'loudness': None, \
              'mode': None,'speechiness': None,'tempo': None,'time_signature': None,'valence': None}
        for key in feat_keys:
            tracks[ii][key] = feat[key]

    return playlist_data, tracks 

## Cache song data 

In [ ]:
songcache={}

In [ ]:
for ii,playlist in enumerate(request_urls):
    if playlist in songcache:
        continue
    try:
        playlist_data, tracks = get_playlist_info(playlist,user = 'spotify')
        all_data = [{**playlist_data,**track} for track in tracks]
        print('Loaded playlist: {}, {}/{}'.format(playlist_data['playlist_name'],ii+1,len(request_urls)))
        songcache[playlist] = all_data
        time.sleep(1)
    except Exception as e:
        print(e)
        time.sleep(5)

## Write to CSV

In [ ]:
df = pd.DataFrame()
for key in songcache:
    play_df = pd.DataFrame(songcache[key])
    df = df.append(play_df)

In [ ]:
df = df.reset_index(drop=True)
df.to_csv('spotify.csv')